# Data Wrangling
* [1 Data Collection](#1_Data_Collection)
    * [1.1 Importing Taxi Trip Datasets](#1.1_Importing_Taxi_Trip_Datasets)
    * [1.2 Concatinating Into One DataFrame](#1.2_Concatinating_Into_One_DataFrame)
    * [1.3 Importing Additional Datasets](#1.3_Importing_Additional_Datasets)
    * [1.4 Data Merging](#1.4_Data_Merging)
    * [1.5 Applying Merge On Zones And Population](#1.5_Applying_Merge_On_All_Datasets)
    * [1.6 Removing Whitespaces](#1.6_Removing_whitespaces)
    * [1.7_Merging_on_boroughs](#1.7_Merging_on_boroughs)
    * [1.8 Merging on Taxi Data](#1.8_Merging_On_Taxi_Data)
* [2 Data Definition](#2_Data_Definition)
    * [2.1 Data Columns](#2.1_Data_Columns)
    * [2.2 Data Types](#2.2_Data_Types)
    * [2.3 Descriptions_Of_Columns](#2.3_Descriptions_Of_Columns)
    * [2.4 Counts_And_Percents](#2.4_Counts_And_Percents)
* [3 Data Cleaning](#3_Data_Cleaning)
    * [3.1 Filling NULL_Values](#3.1_Filling_NULL_values)
    * [3.2_Checking_For_Duplicates](#3.2_Checking_For_Duplicates)
    * [3.3_Finished_Cleaning](#3.3_Finished_Cleaning)

# 1 Data Collection<a id='1_Data_Collection'></a>

In [ ]:
import pandas as pd
import csv
from datetime import datetime

# 1.1  Importing Taxi Trip Datasets <a id='1.1_Importing_Taxi_Trip_Datasets'></a>
The data is stored in a dataset folder in my directory and the data for each month is stored in a separate file. I will be importing each file in its own dataframe and then concatinating them into one big dataframe.

In [ ]:
# The data is stored in a dataset folder in my directory.
# The data for each month is stored in a separate file.
january_data = pd.read_csv('./datasets/yellow_tripdata_2020-01.csv')
february_data = pd.read_csv('./datasets/yellow_tripdata_2020-02.csv')
march_data = pd.read_csv('./datasets/yellow_tripdata_2020-03.csv')
april_data = pd.read_csv('./datasets/yellow_tripdata_2020-04.csv')
may_data = pd.read_csv('./datasets/yellow_tripdata_2020-05.csv')
june_data = pd.read_csv('./datasets/yellow_tripdata_2020-06.csv')

# 1.2 Concatinating Into One DataFrame <a id='1.2_Concatinating_Into_One_DataFrame'></a>
The following cell will combine the entire taxi data into one big dataframe by concating all the monthly data that was imported/

In [ ]:
pandemic_taxi_data = january_data
pandemic_taxi_data = pandemic_taxi_data.append([february_data, march_data, april_data,
                                                may_data, june_data]).reset_index(drop=True)
pandemic_taxi_data.tail()

# 1.3 Importing Additional Datasets<a id='1.3_Importing_Additional_Datasets'></a>

The next two datasets are the taxi zones, which holds data that lets us know where a LocationID is located in the city and a population dataset that lets us know how many people are in each borough.

In [ ]:
taxi_zones = pd.read_csv('./datasets/taxi_zones.csv')
taxi_zones.head(10)

In [ ]:
borough_population = pd.read_csv('./datasets/2020_borough_population.csv')
borough_population

# 1.4. Data Merging <a id='1.4_Data_Merging'></a>

# 1.5 Applying Merge On All Datasets <a id='1.5_Applying_Merge_On_All_Datasets'></a>
The next cells will focus on merging taxi_zones and borough_population based off their boroughs. The idea is to have the columns merge and have a population entry for each borough entry in taxi_zones.

In [ ]:
borough_population['Borough'].unique()

In [ ]:
taxi_zones['borough'].unique()

# 1.6 Removing Whitespaces <a id = '1.6_Removing_whitespaces'></a>
It seems that they have the merging conditions we want but the values of borough_population has their row values with extra whitespace. Data cleaning will need to be used before merging them

In [8]:
# In order to elimimate whitespaces, we must convert the values in the 'Borough' column to strings,
# then we can strip the whitespaces out.
borough_population['Borough'] = borough_population['Borough'].str.strip()
# The borough population data needs to remove the values that do not have the total population of a borough
borough_population = borough_population.loc[borough_population['Age Group'] == 'Total']
borough_population['Borough'].unique()

array(['New York City', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'], dtype=object)

# 1.7 Merging on boroughs <a id='1.7_Merging_on_boroughs'></a>
The next step is to merge taxi_zones and borough_population on their 'borough's. The only columns that will be needed from the taxis data is 'Borough' and '2020'.

In [9]:
# Next step, we merge. We will merge them based off their 'borough' columns.
# Keeping the keys in order of the taxi_zones.
taxi_zones = taxi_zones.merge(borough_population[['Borough','2020']], left_on='borough',
                              right_on='Borough', how='left')
taxi_zones.head()

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough,Borough,2020
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR,NaN,NaN
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens,Queens,2330295.0
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx,Bronx,1446788.0
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan,Manhattan,1638281.0
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island,Staten Island,487155.0


# 1.8 Merging on Taxi ID <a id='1.8_Merging_On_Taxi_Data'></a>

The final merges consists of bringing the LocationID, zones and Borough into the taxi data table.

In [10]:
# The next step is to merge the taxi ride data into the taxi zones
# To avoid 
pandemic_taxi_data = pandemic_taxi_data.merge(taxi_zones[['zone', 'LocationID', 'Borough']],
                                              left_on=['PULocationID'],
                                              right_on=['LocationID'])
pandemic_taxi_data = pandemic_taxi_data.rename(columns={'zone': 'zone_PU',
                                  'LocationID': 'LocationID_PU',
                                  'Borough': 'Borough_PU'})

In [11]:
pandemic_taxi_data = pandemic_taxi_data.merge(taxi_zones[['zone', 'LocationID', 'Borough']],
                                             left_on=['DOLocationID'],
                                             right_on=['LocationID'])
pandemic_taxi_data = pandemic_taxi_data.rename(columns={'zone': 'zone_DO',
                                                       'LocationID': 'LocationID_DO',
                                                       'Borough': 'Borough_DO'})
pandemic_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,zone_PU,LocationID_PU,Borough_PU,zone_DO,LocationID_DO,Borough_DO
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,...,0.0,0.3,11.27,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
1,2.0,2020-01-01 00:24:22,2020-01-01 00:25:54,1.0,0.56,1.0,N,238,239,1.0,...,0.0,0.3,9.00,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
2,2.0,2020-01-01 00:46:46,2020-01-01 00:49:17,1.0,0.62,1.0,N,238,239,1.0,...,0.0,0.3,9.30,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
3,1.0,2020-01-01 00:17:53,2020-01-01 00:22:47,1.0,0.80,1.0,Y,238,239,1.0,...,0.0,0.3,11.15,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan
4,2.0,2020-01-01 00:49:45,2020-01-01 00:54:12,1.0,0.86,1.0,N,238,239,1.0,...,0.0,0.3,11.16,2.5,Upper West Side North,238,Manhattan,Upper West Side South,239,Manhattan


# 1.9 Creating Dummy Variblaes <a id= '1.9Creating_Dummy_Variables'></a>

This will create dummy variables for the Borough_PU and Borough_DP.

In [12]:
Borough_PU = pandemic_taxi_data['Borough_PU']
Borough_DO = pandemic_taxi_data['Borough_DO']

PU_dummies = pd.get_dummies(Borough_PU)
DO_dummies = pd.get_dummies(Borough_DO)

In [13]:
PU_dummies.columns = ['Bronx_PU', 'Brooklyn_PU', 'Manhattan_PU', 'Queens_PU', 'Staten_Island_PU']
DO_dummies.columns = ['Bronx_DO', 'Brooklyn_DO', 'Manhattan_DO', 'Queens_DO', 'Staten_Island_DO']

In [14]:
pandemic_taxi_data = pandemic_taxi_data.join(PU_dummies)
pandemic_taxi_data = pandemic_taxi_data.join(DO_dummies)

# 2. Data Definition <a id='2_Data_Definition'></a>

The following cells will begin to explore what the data holds and what the data looks like.

# 2.1 Data Columns <a id='2.1_Data_Columns'></a>

The names of the Data Columns that are present in the data table.

In [15]:
len(pandemic_taxi_data.columns)

34

In [16]:
for i in range(len(pandemic_taxi_data.columns)):
    print('{}. {}'.format(i + 1, pandemic_taxi_data.columns[i]))

1. VendorID
2. tpep_pickup_datetime
3. tpep_dropoff_datetime
4. passenger_count
5. trip_distance
6. RatecodeID
7. store_and_fwd_flag
8. PULocationID
9. DOLocationID
10. payment_type
11. fare_amount
12. extra
13. mta_tax
14. tip_amount
15. tolls_amount
16. improvement_surcharge
17. total_amount
18. congestion_surcharge
19. zone_PU
20. LocationID_PU
21. Borough_PU
22. zone_DO
23. LocationID_DO
24. Borough_DO
25. Bronx_PU
26. Brooklyn_PU
27. Manhattan_PU
28. Queens_PU
29. Staten_Island_PU
30. Bronx_DO
31. Brooklyn_DO
32. Manhattan_DO
33. Queens_DO
34. Staten_Island_DO


# Do your column names correspond to what those columns store? 

The column names appeaer to each describe what each column holds. Each column is described as: 
1. VendorID: A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc. : The date and time when the meter was engaged.
2. tpep_pickup_datetime: The date and time when the meter was engaged.
3. tpep_dropoff_datetime: The date and time when the meter was disengaged.
4. passenger_count: The number of passengers in the vehicle. This is a driver-entered value.
5. trip_distance: The elapsed trip distance in miles reported by the taximeter.
6. RatecodeID:The final rate code in effect at the end of the trip. 1= Standard rate 2=JFK 3=Newark 4=Nassau or Westchester 5=Negotiated fare 6=Group ride
7. store_and_fwd_flag: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip N= not a store and forward trip
8. PULocationID: TLC Taxi Zone in which the taximeter was engaged
9. DOLocationID: TLC Taxi Zone in which the taximeter was disengaged
10. payment_type: A numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided trip
11. fare_amount: The time-and-distance fare calculated by the meter.
12. extra: Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.
13. mta_tax: 0.50 cents MTA tax that is automatically triggered based on the metered rate in use.
14. tip_amount: Tip amount – This field is automatically populated for credit card tips. Cash tips are not included.
15. tolls_amount: Total amount of all tolls paid in trip.
16. improvement_surcharge: 0.30 cents improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
17. total_amount: The total amount charged to passengers. Does not include cash tips.
18. congestion_surcharge: 
19. zone_PU: The area zone of where the pick up location was.
20. LocationID_PU: The taxi zone number of where the pick up location was.
21. Borough_PU: The borough of the pick up location.
22. zone_DO: The area zone of where the pick up location was.
23. LocationID_DO: The taxi zone number of where the pick up location was.
24. Borough_DO: The borough of the pick up location.

# 2.2 Data Types <a id='2.2_Data_Types'></a>

The next part is going to explore what the data types of each column. This will help gain a better understanding of what we may need to clean and how to clean it.

In [17]:
pandemic_taxi_data.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
zone_PU                   object
LocationID_PU              int64
Borough_PU                object
zone_DO                   object
LocationID_DO              int64
Borough_DO                object
Bronx_PU                   uint8
Brooklyn_PU                uint8
Manhattan_PU               uint8
Queens_PU                  uint8
Staten_Island_PU           uint8
Bronx_DO                   uint8
Brooklyn_D

The .dtypes method shows and the datatypes of 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type',
'zone_PU', 'LocationID_PU', 'Borough_PU', 'zone_DO','LocationID_DO', and 'Borough_DO' are supposed to be categorical but are marked as in64 objects. The next cells will convert the categorical columns to a category object.

In [18]:
# Iterate through the columns and change them into category types
list_of_categorical = ['store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type',
                      'zone_PU', 'LocationID_PU', 'Borough_PU',
                      'zone_DO', 'LocationID_DO', 'Borough_DO']
for item in list_of_categorical:
    pandemic_taxi_data[item] = pandemic_taxi_data[item].astype('category')

In [19]:
pandemic_taxi_data.dtypes

VendorID                  float64
tpep_pickup_datetime       object
tpep_dropoff_datetime      object
passenger_count           float64
trip_distance             float64
RatecodeID                float64
store_and_fwd_flag       category
PULocationID             category
DOLocationID             category
payment_type             category
fare_amount               float64
extra                     float64
mta_tax                   float64
tip_amount                float64
tolls_amount              float64
improvement_surcharge     float64
total_amount              float64
congestion_surcharge      float64
zone_PU                  category
LocationID_PU            category
Borough_PU               category
zone_DO                  category
LocationID_DO            category
Borough_DO               category
Bronx_PU                    uint8
Brooklyn_PU                 uint8
Manhattan_PU                uint8
Queens_PU                   uint8
Staten_Island_PU            uint8
Bronx_DO      

The appropriate datatypes have been changed from int64s into categorical data types.

# 2.3 Description Of Columns <a id= '2.3_Descriptions_Of_Columns'></a>

The next cell will describe what the columns statistical values look like. It'll give us an idea of how the data is behaving.

In [20]:
pandemic_taxi_data.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,...,Bronx_PU,Brooklyn_PU,Manhattan_PU,Queens_PU,Staten_Island_PU,Bronx_DO,Brooklyn_DO,Manhattan_DO,Queens_DO,Staten_Island_DO
count,1.639500e+07,1.639500e+07,1.667322e+07,1.639500e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,...,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07,1.667322e+07
mean,1.664669e+00,1.491998e+00,3.073729e+00,1.043600e+00,1.253740e+01,1.133242e+00,5.230166e-01,2.132019e+00,3.209923e-01,2.979066e-01,...,3.664799e-03,1.332784e-02,9.207748e-01,6.203829e-02,1.057984e-04,9.930956e-03,4.272804e-02,8.926352e-01,5.258733e-02,3.726335e-04
std,4.721061e-01,1.136433e+00,1.656767e+02,6.765441e-01,1.954438e+02,1.224568e+02,1.224504e+02,2.600147e+00,1.635388e+00,3.473591e-02,...,6.042656e-02,1.146744e-01,2.700899e-01,2.412251e-01,1.028529e-02,9.915812e-02,2.022433e-01,3.095764e-01,2.232082e-01,1.930012e-02
min,1.000000e+00,0.000000e+00,-3.062000e+01,1.000000e+00,-1.259000e+03,-2.700000e+01,-5.000000e-01,-4.932200e+02,-4.000000e+01,-3.000000e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,9.800000e-01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.620000e+00,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.920000e+00,0.000000e+00,3.000000e-01,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,2.000000e+00,2.000000e+00,3.000000e+00,1.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.860000e+00,0.000000e+00,3.000000e-01,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,2.000000e+00,9.000000e+00,2.970045e+05,9.900000e+01,6.711001e+05,5.000008e+05,5.000005e+05,1.100000e+03,9.117500e+02,3.000000e-01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


# 2.4 Counts and Percents Unique Values <a id='2.4_Counts_And_Percents'></a>

The following cell will count the cardinality of each column and see how their ratios are.

In [21]:
for item in pandemic_taxi_data.columns:
    len_unique = len(pandemic_taxi_data[item].unique())
    print('The cardinality of {} is: {}'.format(item, len_unique))  

The cardinality of VendorID is: 3
The cardinality of tpep_pickup_datetime is: 6336458
The cardinality of tpep_dropoff_datetime is: 6342811
The cardinality of passenger_count is: 11
The cardinality of trip_distance is: 5544
The cardinality of RatecodeID is: 8
The cardinality of store_and_fwd_flag is: 3
The cardinality of PULocationID is: 258
The cardinality of DOLocationID is: 258
The cardinality of payment_type is: 6
The cardinality of fare_amount is: 8682
The cardinality of extra is: 78
The cardinality of mta_tax is: 16
The cardinality of tip_amount is: 3932
The cardinality of tolls_amount is: 1461
The cardinality of improvement_surcharge is: 3
The cardinality of total_amount is: 13972
The cardinality of congestion_surcharge is: 10
The cardinality of zone_PU is: 258
The cardinality of LocationID_PU is: 258
The cardinality of Borough_PU is: 6
The cardinality of zone_DO is: 258
The cardinality of LocationID_DO is: 258
The cardinality of Borough_DO is: 6
The cardinality of Bronx_PU is: 2

# 3. Data Cleaning <a id='3_Data_Cleaning'></a>

This section will cover how I clean the data's nan values, outliers and checking for duplicates.

In [22]:
pandemic_taxi_data.isna().sum()

VendorID                 278223
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          278223
trip_distance                 0
RatecodeID               278223
store_and_fwd_flag       278223
PULocationID                  0
DOLocationID                  0
payment_type             278223
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
zone_PU                       0
LocationID_PU                 0
Borough_PU                 1475
zone_DO                       0
LocationID_DO                 0
Borough_DO                29108
Bronx_PU                      0
Brooklyn_PU                   0
Manhattan_PU                  0
Queens_PU                     0
Staten_Island_PU              0
Bronx_DO                      0
Brooklyn_DO                   0
Manhatta

In [ ]:
col_with_null_vals = ['VendorID', 'passenger_count', 'RatecodeID',
                      'payment_type', 'Manhattan_PU', 'Queens_PU', 'Bronx_PU', 'Brooklyn_PU',
                     'Staten_Island_PU', 'Manhattan_DO', 'Queens_DO', 'Bronx_DO', 'Brooklyn_DO',
                     'Staten_Island_DO']

df_nulls = pandemic_taxi_data[col_with_null_vals].isnull()
df_nulls.columns = ['V', 'p', 'R', 'p_type', 'Man_PU', 'Q_PU', 'BX_PU', 'BK_PU', 'SI_PU',
                   'Man_DO', 'Q_DO', 'BX_DO', 'BK_DO', 'SI_DO']

df_nulls.to_csv('./datasetsTFNullTable.csv')

# 3.1 Filling NULL values <a id='3.1_Filling_NULL_values'></a>

This lets us know that there are seven columns with NaN values. What I'll do with the NaN values is substitute the NaNs with 0s

In [ ]:
pandemic_taxi_data.tail()

In [ ]:
a = (pandemic_taxi_data['tpep_dropoff_datetime'].astype('datetime64') - 
 pandemic_taxi_data['tpep_pickup_datetime'].astype('datetime64'))

In [ ]:
mins = [(a[i].components[1] * 60) + a[i].components[2] + (round(a[i].components[3] / 60, 2)) for i in range(len(a))]
mins

In [ ]:
pandemic_taxi_data.columns

In [ ]:
pandemic_taxi_data.index

In [ ]:
pandemic_taxi_data['Mins_In_Ride'] = mins
pandemic_taxi_data.index = pandemic_taxi_data['tpep_pickup_datetime']
pandemic_taxi_data = pandemic_taxi_data.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)
pandemic_taxi_data.tail()

In [ ]:
pandemic_taxi_data['VendorID'] = pandemic_taxi_data['VendorID'].fillna(0)
pandemic_taxi_data['passenger_count'] = pandemic_taxi_data['passenger_count'].fillna(0)
pandemic_taxi_data['RatecodeID'] = pandemic_taxi_data['RatecodeID'].fillna(0)

In [ ]:
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].astype('string')
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].fillna('NK') # Not Known
pandemic_taxi_data['store_and_fwd_flag'] = pandemic_taxi_data['store_and_fwd_flag'].astype('category')

pandemic_taxi_data['payment_type'] = pandemic_taxi_data['payment_type'].fillna(5) # For Unknown

pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].astype('string')
pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].fillna('Unknown')
pandemic_taxi_data['Borough_PU'] = pandemic_taxi_data['Borough_PU'].astype('category')

pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].astype('string')
pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].fillna('Unknown')
pandemic_taxi_data['Borough_DO'] = pandemic_taxi_data['Borough_DO'].astype('category')

In [ ]:
pandemic_taxi_data.isna().sum()

# 3.2 Checking For Outliers <a id='3.2_Checking_For_Duplicates'></a>

Any duplicates based off VendorID, trip_distance, total_amount should be dropped

In [ ]:
pandemic_taxi_data.head()

Time to remove wrongly inputted data

In [ ]:
list_of_dates = pandemic_taxi_data.index
list_of_dates = list(map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), list_of_dates))
list_of_dates

In [ ]:
# There appears to be the years 2008, 2009, 2003, and 2019 and erroneous dates in here so I will remove them.
pandemic_taxi_data.index = list_of_dates
indicies = []
for i in range(len(pandemic_taxi_data)):
    if pandemic_taxi_data.index[i].year != 2020 or pandemic_taxi_data.index[i].month > 6:
        indicies.append(pandemic_taxi_data.index[i])
indicies

In [ ]:
pandemic_taxi_data = pandemic_taxi_data.drop(indicies, axis=0)

In [ ]:
# Time to remove outliers before resampling them into groups of days
pandemic_taxi_data.describe().T

In [ ]:
len(pandemic_taxi_data)

In [ ]:
bad_columns = ['trip_distance', 'RatecodeID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
               'tolls_amount', 'improvement_surcharge', 'total_amount', 'Mins_In_Ride']
for column in bad_columns:
    mn = pandemic_taxi_data[column].mean()
    std = pandemic_taxi_data[column].std()
    cut_off = 3 * std
    lower, upper = mn - cut_off, mn + cut_off
    pandemic_taxi_data = pandemic_taxi_data[(pandemic_taxi_data[column] < upper) & (pandemic_taxi_data[column] > lower)]

In [ ]:
# Drop rows in trip_distance and fare amount that are less than 0
pandemic_taxi_data = pandemic_taxi_data[pandemic_taxi_data['trip_distance'] > 0]
pandemic_taxi_data = pandemic_taxi_data[pandemic_taxi_data['fare_amount'] > 0]
len(pandemic_taxi_data)

In [ ]:
pandemic_taxi_data.describe().T

In [ ]:
# next task is to group the index by hour
pandemic_taxi_data = pandemic_taxi_data.resample('H').mean()
pandemic_taxi_data.head()

In [ ]:
# Last step is to make a random walk of the total_amount. In other words, make the next entiries' total_amount
# value be dependent of the current one
shifted_values = pandemic_taxi_data[['total_amount']]
shifted_values = shifted_values.shift(1, axis=0)
pandemic_taxi_data['total_amount'] = shifted_values
pandemic_taxi_data['total_amount'].head()

# 3.3 Finished Cleaning <a id='3.3_Finished_Cleaning'></a>

In [ ]:
pandemic_taxi_data.to_csv('./datasets/Clean_Pandemic_Taxi_Data.csv')